In [36]:
from transformers import pipeline
import torch
from DEMO.recycle.utils import txt_reader, regex_search
import os 
# set gpu 5-6-7 is available


In [37]:
# from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# model_8bit = AutoModelForCausalLM.from_pretrained(
#     "meta-llama/Meta-Llama-3-8B-Instruct", 
#     quantization_config=quantization_config,
#     device_map = 'auto'
# )


In [38]:
# set up the model
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "microsoft/Phi-3-mini-128k-instruct"

pipe = pipeline("text-generation",
                model = model_id,
                # device = 7,
                device_map = 'auto', 
                torch_dtype=torch.float16,
                trust_remote_code=True, 
                do_sample=True, 
                temperature=1)
raw_documents = txt_reader('/mnt/HDD1/duclv/ACM-AMM/database/uploaded_docs/Earthquakes Conversation.txt')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [56]:
import re
from docx import Document
import time
# Create a new Document
doc = Document()
current_time = str(time.strftime("%Y%m%d-%H%M%S"))
saving_folder = "./conversation/"

chat_history = []
number_of_turns = 0
chat_history.append('initialize')
# write a formal, detailed and informative meeting minutes

In [57]:

question = str(input("Enter your question: "))
# convert chat history to string
_chat_history = ' '.join(chat_history)
# print(_chat_history)

user_intent_prompt = [
    {
        "role": "system",
        "content": f"You are an expert in understanding conversation.   \
                    base on the chat history below, you should try to identify \
                    the user's intent, the answer should be not too long and keep the original meaning\
                   ",
    },  
    {"role": "user", "content": f"  based on chat history: ({chat_history}) \|\n\
                                    and current question: ({question}). \| \
                                    understand the the current user's question or command \
                                    and you can paraphrase it to make it easier understand for LLM if needed.\
                                    the user's intent must be in between <intent> and </intent>\
                                        "},
]

user_intent = ''
# user_intent = pipe(user_intent_prompt, max_new_tokens=500)[0]['generated_text'][-1]['content']

# user_intent = regex_search( user_intent)

    
# print('user_intent:', user_intent)
prompt = f"Answer the question and using the context and user below if needed.\
            \|Context:  {raw_documents} \| \
            \|Question: {question}\ |\
              Answer:"

messages = [
    {
        "role": "system",
        "content": f"You are an expert in meeting minutes and question answering. \
                    Answer base on the context and the chat history,  \
                    , not redundant and follow the conversation flow.\
                    Do not provide any information that is not in the context or the chat history.\
                    if you need more information, you can ask questions.\
                    You should NOT answer the information that user did NOT ask. \
                    You must ask questions to get more information if needed. \
                        ",

    },  
    {"role": "user", "content": f"{prompt}"},
    
]
# messages = [
#     {
#         "role": "system",
#         "content": f"I'm your AI assistant for meetings and conversations.\
#                     Based on the chat history you provide, I can help you with: \
#                     Meeting Minutes: I can summarize key points, decisions, and action items. \n \
#                     Question Answering: I can answer your questions about the conversation, \n \
#                     but I'll stick to what's been discussed and ask for clarification if needed. \n \
#                     Let's get started! Paste the chat history below, and I'll do my best to assist you. "
#     },  
#     {"role": "user", "content": f"{prompt}"},
    
# ]
doc.add_paragraph(f"turn {number_of_turns}", style='Heading 1')
doc.add_paragraph("user question: {} \n".format(question), style='Body Text')

response = pipe(messages, max_new_tokens=1024)[0]['generated_text'][-1]
doc.add_paragraph("Meeting minutes: {} \n".format(response['content']), style='Body Text')
print(type(response))
# print(response['content'])  # Print the assistant's response
number_of_turns += 1

# prompt = f"Answer the question and using the context and user below if needed.\
#             \|Context:  {response['content']} \| \
#             \|Question: select a list of provided information in this meeting minutes\
#             that should be double checked. The information are Number of some specific information, \
#             What did someone say about something. \
#             . Return in a LIST \n \
#             For example: 1. the date is 07-08-2020, \
#             , the list of attendees is a and b\ \n \
#             Answer:"
# messages = [
#     {
#         "role": "system",
#         "content": f"You are an expert in extract information in a meeting minutes \
#                         ",

#     },  
#     {"role": "user", "content": f"{prompt}"},
    
# ]
# infor_response = pipe(messages, max_new_tokens=1024)[0]['generated_text'][-1]
# print("------------information-------------")
# # print(response['content'])  # Print the assistant's response



# chat_history.append(f"|Turn {number_of_turns}, My question: {question},\
#                         Your response : {response['content']} |")

# ######

# # doc.add_paragraph("user intent: {}\n".format(user_intent))
# doc.add_paragraph("\nKey informations: {}\n".format(infor_response['content']))


prompt = f"Answer the question and using the context and user below if needed.\
            \|Context:  {response['content']} \| \
            \|Question: Create verification Wh-questions for each mentioned information in above meeting minutes. \
            Focus on the information that should be double checked, number of some specific information, what did someone say about something. \
            Return in a list of answer only. \n \
            Answer:"
messages = [
    {
        "role": "system",
        "content": f"You are an expert in making questions \
                        ",

    },  
    {"role": "user", "content": f"{prompt}"},
    
]
question_response = pipe(messages, max_new_tokens=1024)[0]['generated_text'][-1]
doc.add_paragraph("\nQuestion: {} \n".format(question_response['content']), style='Body Text')


prompt = f"Answer the question and using the context and user below if needed.\
            \|Context:  {raw_documents} \| \
            \|Questions: {question_response['content']}\ |\
            \|Question: Answer the above questions based on the context \
            Return in a list of answer as a completion sentence \n \
            Answer:"
messages = [
    {
        "role": "system",
        "content": f"You are an expert in answer the questions based on provided meeting conversation \
                        ",

    },  
    {"role": "user", "content": f"{prompt}"},
    
]
answer_response = pipe(messages, max_new_tokens=1024)[0]['generated_text'][-1]
doc.add_paragraph("\n Answer: {} \n".format(answer_response['content']), style='Body Text')


prompt = f"Answer the question and using the context and user below if needed.\
            \|Original meeting minutes:  {response['content']} \| \
            \|correct answer: {answer_response['content']}\ |\
            \|Question: Based on the original meeting minutes and the correct answers, \
             write a new version of the meeting minutes. \
             if there are conflicting information, should use the correct answer. \| \n \
            Tell me the changes you made in the new version if you did. \
             Answer: "

messages = [
    {
        "role": "system",
        "content": f"You are an expert in writing meeting minutes \
                        ",

    },  
    {"role": "user", "content": f"{prompt}"},
    
]
answer_response = pipe(messages, max_new_tokens=1024)[0]['generated_text'][-1]
doc.add_paragraph("\nNew version: {} \n".format(answer_response['content']), style='Body Text')


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<class 'dict'>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [58]:
chat_history

['initialize']

In [59]:
import os
if os.path.exists(saving_folder) == False:
    os.mkdir(saving_folder)
doc_name = saving_folder + "conversation_" + current_time + ".docx"

doc.save(doc_name)

In [8]:
from huggingface_hub import InferenceClient

client = InferenceClient("http://127.0.0.1:8080")
for token in client.text_generation("How do you make cheese?", max_new_tokens=12, stream=True):
    print(token)

# To
# make
# cheese
#,
# you
# need
# to
# start
# with
# milk
#.

ConnectionError: (MaxRetryError("HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f93e6c16250>: Failed to establish a new connection: [Errno 111] Connection refused'))"), '(Request ID: c70a4b59-6b15-47e1-a3cd-e05184b4aeee)')

In [ ]:
from transformers import pipeline, Conversation

chatbot = pipeline(model="facebook/blenderbot-400M-distill")
# Conversation objects initialized with a string will treat it as a user message
conversation = "I'm looking for a movie - what's your favourite one?"
conversation = chatbot(conversation)
print(conversation)

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain_huggingface import HuggingFacePipeline
from langchain_community.document_loaders import DirectoryLoader
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# set OPENAI_API_KEY environment variable to your OpenAI API key

DOCS_DIR_PATH = "database/uploaded_docs"
model_id = "microsoft/Phi-3-mini-128k-instruct"
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, device_map ='auto')
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500, return_full_text=False, temperature=1.0, do_sample=True
)
hf = HuggingFacePipeline(pipeline=pipe)
llm = hf
# Load raw documents from the directory
raw_documents = DirectoryLoader(os.path.abspath(DOCS_DIR_PATH)).load()
### Construct retriever ###
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
print(raw_documents)
docs = raw_documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceHubEmbeddings())
retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = """You are an assistant for question-answering tasks. \ """
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
If you don't know the answer, just say that you don't know. \
Do not synthesize text that is not relevant to the question. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
response = conversational_rag_chain.invoke(
    {"input": "what can you do?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

In [ ]:
# print(response['answer'].split('[Response]:')[-1].split('AI:')[-1])
print(response['answer'])

In [ ]:
print(response['chat_history'])

In [ ]:
print(response['input'])

In [1]:
import streamlit as st

animal_shelter = ['cat', 'dog', 'rabbit', 'bird']

animal = st.text_input('Type an animal')

if st.button('Check availability'):
    have_it = animal.lower() in animal_shelter
    'We have that animal!' if have_it else 'We don\'t have that animal.'

2024-06-28 23:43:19.100 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-06-28 23:43:19.196 
  command:

    streamlit run /home/duclv/.conda/envs/mm/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [7]:
from diffusers import StableDiffusionInpaintPipeline
import torch
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    revision="fp16",
    torch_dtype=torch.float16,
    
)

/home/duclv/.conda/envs/mm/lib/python3.11/site-packages/diffusers/pipelines/pipeline_loading_utils.py:219: FutureWarning: You are loading the variant fp16 from runwayml/stable-diffusion-inpainting via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(
safety_checker/model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /home/duclv/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/afeee10def38be19995784bcc811882409d066e5/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /home/duclv/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/afeee10def38be19995784bcc811882409d066e5/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /home/duclv/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/afeee10def38be19995784bcc811882409d066e5/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /home/duclv/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/afeee10def38be19995784bcc811882409d066e5/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/home/duclv/.conda/envs/mm/lib/python3.11/

In [8]:
prompt = "Face of a yellow cat, high resolution, sitting on a park bench"
#image and mask_image should be PIL images.
import PIL
# download the image
import requests
from PIL import Image
from io import BytesIO

img1 = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
img2 = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"
img1 = requests.get(img1)
img2 = requests.get(img2)
image = Image.open(BytesIO(img1.content))
mask_image = Image.open(BytesIO(img2.content))

#The mask structure is white for inpainting and black for keeping as is
image = pipe(prompt=prompt, image=image, mask_image=mask_image).images[0]
image.save("./yellow_cat_on_park_bench.png")

  0%|          | 0/50 [00:00<?, ?it/s]